In [1]:
#!/usr/bin/env python3

import sys
sys.path.insert(0, '../pyaf/py_aff3ct/build/lib')

import numpy as np
import py_aff3ct as aff3ct
import math
import time
import matplotlib.pyplot as plt
from datetime import timedelta
from tempfile import TemporaryFile

from py_aff3ct.module.py_module import Py_Module
import py_aff3ct.tools.frozenbits_generator as tool_fb
import py_aff3ct.tools.noise as tool_noise
import py_aff3ct.tools.sparse_matrix as tool_sp
import random 

In [2]:
class source(Py_Module):
	# 
	# out : vec_cnt -> Où est-on dans le vecteur ? 
	# 		r_in	-> initial vector : never modified
	#		enable	-> Always true
	#		delta_x -> From -4 to 4
	#		delta_y -> From -4 to 4
	#		ix_x	-> Constant for now
	#		ix_y	-> Constant for now
	#		x		-> Position in the image along the X-axis
	#		y		-> Position in the image along the Y-axis
	#
	def generate(self, r_in, enable, delta_x, delta_y, ix_x, ix_y, x, y):
		r_in[:] = self.r_in[:]
		enable 	= self.enable
		ix_x[:] = self.ix_x[:]
		ix_y[:] = self.ix_y[:]
		if (enable):
			x[:] 		 = self.vec_cnt // len(self.delta_y_range)
			y[:] 		 = self.vec_cnt %  len(self.delta_y_range)
			delta_x[:] 	 = self.delta_x_range[x]	# Looks like : [[number]] with number = {-4 ; 4}
			delta_y[:] 	 = self.delta_y_range[y]	# Looks like : [[number]] with number = {-4 ; 4}
			self.vec_cnt = self.vec_cnt + 1 	
			if(self.vec_cnt >= pow(len(self.delta_y_range),2)):
				enable = 0	# Tout les données ont été générées
		return 0
    
	def __init__(self, r_in, N, ix_x, ix_y): # N == len(r_in[0,:])

		Py_Module.__init__(self) # Call the aff3ct Py_Module __init__
		
		self.name   		= "source_xy"   		# Set your module's name
		self.r_in   		= r_in					# Constant
		self.delta_x_range 	= np.arange(-4,4,0.05)
		self.delta_y_range 	= np.arange(-4,4,0.05)
		self.enable 		= 1
		self.ix_x   		= ix_x # np.random.randint(N)  # x position in the vector
		self.ix_y   		= ix_y # self.ix_x + 5 % N		# y position in the vector
		self.x				= 0						# Position in the image along x
		self.y				= 0						# Position in the image along y
		self.vec_cnt 		= 0 					# counter for delta_x and delta_y
		self.delta_x 		= np.ndarray(shape = (1,1), dtype = np.float32)
		self.delta_y		= np.ndarray(shape = (1,1), dtype = np.float32)

		t_generate = self.create_task("generate") 	# create a task for your module

		s_r_in    = self.create_socket_out  (t_generate, "r_in"   , N		 , np.float32 ) # create an output socket for the task t_generate
		s_enable  = self.create_socket_out  (t_generate, "enable" , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_delta_x = self.create_socket_out  (t_generate, "delta_x", 1		 , np.float32 )	# create an output socket for the task t_generate
		s_delta_y = self.create_socket_out  (t_generate, "delta_y", 1	 	 , np.float32 ) # create an output socket for the task t_generate
		s_ix_x    = self.create_socket_out  (t_generate, "ix_x"   , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_ix_y    = self.create_socket_out  (t_generate, "ix_y"   , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_x    	  = self.create_socket_out  (t_generate, "x"      , 1		 , np.int32   ) # create an output socket for the task t_generate
		s_y    	  = self.create_socket_out  (t_generate, "y"   	  , 1		 , np.int32   ) # create an output socket for the task t_generate

		self.create_codelet(t_generate, lambda slf, lsk, fid: slf.generate(lsk[s_r_in],lsk[s_enable],lsk[s_delta_x],lsk[s_delta_y],lsk[s_ix_x],lsk[s_ix_y],lsk[s_x], lsk[s_y]))

In [3]:
#############
# Main script
#############


########
# Parameters


#N = 576 
#K = 288
# K = 64*2
# N = 3*K
N = 128
K = 64

ebn0_min = 2.7
ebn0_max = 2.71
ebn0_step = 0.2

ebn0 = np.arange(ebn0_min,ebn0_max,ebn0_step)
esn0 = ebn0 + 10 * math.log10(K/N)
sigma_vals = 1/(math.sqrt(2) * 10 ** (esn0 / 20))

delta_x_range = np.arange(-4,4,0.05)
delta_y_range = np.arange(-4,4,0.05)

W = len(delta_x_range)
H = len(delta_y_range)

code_type = "polar"

if(code_type == "turbo"):
	# Build DVBS-RCS2 Turbo encoder.
	enc_n = aff3ct.module.encoder.Encoder_RSC_DB(K,2*K,standard='DVB-RCS1')
	enc_i = aff3ct.module.encoder.Encoder_RSC_DB(K,2*K,standard='DVB-RCS1')

	# Build DVBS-RCS2 Interleaver.
	itl_core = aff3ct.tools.interleaver_core.Interleaver_core_ARP_DVB_RCS1(K//2)
	itl_bit  = aff3ct.module.interleaver.Interleaver_int32(itl_core)
	itl_llr  = aff3ct.module.interleaver.Interleaver_float(itl_core)
	enc = aff3ct.module.encoder.Encoder_turbo_DB(K,N,enc_n,enc_i,itl_bit)

	# Build DVBS-RCS2 Trubo decoder.
	trellis_n = enc_n.get_trellis()
	trellis_i = enc_i.get_trellis()

	dec_n = aff3ct.module.decoder.Decoder_RSC_DB_BCJR_DVB_RCS1(K,trellis_n)
	dec_i = aff3ct.module.decoder.Decoder_RSC_DB_BCJR_DVB_RCS1(K,trellis_i)
	dec   = aff3ct.module.decoder.Decoder_turbo_DB(K,N,8,dec_n,dec_i,itl_llr)

elif(code_type == "polar"): #polar

	fbgen = tool_fb.Frozenbits_generator_GA_Arikan(K, N)
	esn0 = 3.0 + 10 * math.log10(K/N)
	sigma = 1/(math.sqrt(2) * 10 ** (esn0 / 20))
	noise = tool_noise.Sigma(sigma)
	fbgen.set_noise(noise)
	frozen_bits = fbgen.generate()

	enc  = aff3ct.module.encoder.Encoder_polar_sys      (K,N,frozen_bits)     # Build the encoder
	#dec  = aff3ct.module.decoder.Decoder_polar_SC_fast_sys(K,N,frozen_bits)   # Build the decoder
	#dec = aff3ct.module.decoder.Decoder_polar_SCL_MEM_fast_sys(K,N,8,frozen_bits)
	I = 10
	dec = aff3ct.module.decoder.Decoder_polar_SCAN_naive_sys(K, N, I, frozen_bits)
elif(code_type == "ldpc"):
	I = 10
	# H   = tool_sp.alist.read("../py_aff3ct/lib/aff3ct/conf/dec/LDPC/WIMAX_288_576.alist")
	# H   = tool_sp.alist.read("../py_aff3ct/lib/aff3ct/conf/dec/LDPC/10GBPS-ETHERNET_1723_2048.alist")
	H = tool_sp.alist.read("../py_aff3ct/lib/aff3ct/conf/dec/LDPC/CCSDS_64_128.alist")
	

	N   = H.shape[0]
	m   = H.shape[1]
	K   = N - m
	enc  = aff3ct.module.encoder.Encoder_LDPC_from_H    (K, N, H)                                                   # Build the encoder
	dec  = aff3ct.module.decoder.Decoder_LDPC_BP_horizontal_layered_inter_NMS (K, N, I, H, enc.get_info_bits_pos()) # Build the decoder
else:
	K = 106
	N = 127
	t = 3
	poly = aff3ct.tools.BCH_polynomial_generator(N, t)
	enc = aff3ct.module.encoder.Encoder_BCH(K, N, poly)
	dec = aff3ct.module.decoder.Decoder_BCH_std(K, N, poly)

src  = aff3ct.module.source.Source_random_fast(K, 12)
mdm  = aff3ct.module.modem.Modem_BPSK_fast(N)

gen = aff3ct.tools.Gaussian_noise_generator_implem.FAST
chn = aff3ct.module.channel.Channel_AWGN_LLR(N, gen)
mnt = aff3ct.module.monitor.Monitor_BFER_AR(K, 1)

sigma      = np.ndarray(shape = (1,1),  dtype = np.float32)
delta_x    = np.ndarray(shape = (1,1),  dtype = np.float32)
delta_y    = np.ndarray(shape = (1,1),  dtype = np.float32)
ix_x       = np.ndarray(shape = (1,1),  dtype = np.int32  )
ix_y       = np.ndarray(shape = (1,1),  dtype = np.int32  )
noisy_vec  = np.ndarray(shape = (1,N),  dtype = np.float32)
enable     = np.ndarray(shape = (1,1),  dtype = np.int32  )

enable[:] = 0

map = np.zeros( (H,W,3), dtype=np.uint8)
src.reset()

from datetime import datetime

seed = random.randint(0, 123456789)
src.set_seed(seed+1)
chn.set_seed(seed)

Séquence 1 : génération des vecteur "r_in" et "x"

In [4]:
src["generate   ::U_K  "] = enc["encode       ::U_K "]
enc["encode     ::X_N  "] = mdm["modulate     ::X_N1"]
mdm["modulate   ::X_N2 "] = chn["add_noise    ::X_N "]
chn["add_noise  ::Y_N  "] = mdm["demodulate   ::Y_N1"]
mdm["demodulate ::Y_N2 "] = dec["decode_siho  ::Y_N "]
src["generate   ::U_K  "] = mnt["check_errors ::U   "]
dec["decode_siho::V_K  "] = mnt["check_errors ::V   "]
chn["add_noise  ::CP   "].bind( sigma  )
mdm["demodulate ::CP   "].bind( sigma  )

seq1 = aff3ct.tools.sequence.Sequence(src("generate"),  1)
seq1.export_dot("chaos.dot")

The use of parenthesis for accessing tasks is deprecated, use brackets instead.


Chargement du fichier

In [5]:
load_file = False
if(load_file == False):
	for i in range(len(sigma_vals)):
		sigma[:] = sigma_vals[i]
		seq1.exec()		
		print("be=",mnt.get_n_be())
		print("fe=",mnt.get_n_fe())
		mnt.reset()
	with open('noisy_cw.npy', 'wb') as f:
		np.save(f, dec['decode_siho::Y_N'][:])
		np.save(f, src['generate::U_K'][:])
else:
	with open('noisy_cw.npy', 'rb') as f:
		mdm["demodulate ::Y_N2 "][:] = np.load(f)
		src['generate :: U_K'][:] = np.load(f)

r_in = dec['decode_siho::Y_N'][:]

be= 8
fe= 1


Déclaration du module "source".

In [6]:
# Module declaration for sequence 2

ix_x[:]   = np.random.randint(N) #idx[0]
ix_y[:]   = (ix_x[:]+5) % N      #idx[1]

my_src = source(r_in, N, ix_x, ix_y)

Toujours réinitialiser le module "my_src" avant d'effectuer les tests (pour remettre le compteur interne du module à zéro).

In [7]:
for x in range(len(delta_x_range)):
	for y in range(len(delta_y_range)):
		delta_x[:] = delta_x_range[x]
		delta_y[:] = delta_y_range[y]
		my_src['generate'].exec()

		assert((my_src["generate :: r_in"   ][0,:] 	== r_in[0,:]).all()	)
		assert( my_src["generate :: delta_x"][:] 	== delta_x[:]		)
		assert( my_src["generate :: delta_y"][:] 	== delta_y[:]		)
		assert( my_src["generate :: ix_x"   ][:] 	== ix_x[:]			)
		assert( my_src["generate :: ix_y"   ][:] 	== ix_y[:]			)
		assert( my_src["generate :: x"      ][:] 	== x				)
		assert( my_src["generate :: y"      ][:] 	== y				)
print("Module de source validé !\n")

Module de source validé !

